In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_57755/3522731578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# 4-Team Tournament EV Model, with Conference Winner EV and Championship EV

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0.28, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0.72, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, same_conference_teams, other_conference_teams):
    # Conference Winner EV
    # 100% win for this team
    conference_winner_ev = payouts[winning_team]['conference']  # 100% win for this team

    # for Team X in other conference:
    # += (Probability of Team X Winning Conferene) * (Conference Winner Potential Payout for Team X)
    for team in other_conference_teams:
        conference_winner_ev += (
            team_probabilities[team]['semifinal_prob'] *
            payouts[team]['conference']
        )
    
    # Championship EV
    # 100% probability this team reaches championship
    # for Team X in other conference:
    # += (Prob of this team beating Team X in Champ.) * (Champ. Payout for this team) + (Prob. of Team X beating this team in Champ.) * (Champ. Payout for Team X)
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    # Total Portfolio EV
    return conference_winner_ev + championship_ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
scenarios = [
    ("kansas city chiefs", conference_a, conference_b),
    ("buffalo bills", conference_a, conference_b),
    ("washington commanders", conference_b, conference_a),
    ("philadelphia eagles", conference_b, conference_a),
]

ev_results = {}
for winner, same_conference, other_conference in scenarios:
    ev = calculate_portfolio_ev(winner, same_conference, other_conference)
    ev_results[f"{winner.title()} wins semifinal"] = ev

# print EV results
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:.2f}")


Portfolio EV if Kansas City Chiefs wins semifinal: $312355.02
Portfolio EV if Buffalo Bills wins semifinal: $148740.75
Portfolio EV if Washington Commanders wins semifinal: $341796.98
Portfolio EV if Philadelphia Eagles wins semifinal: $196374.03


In [4]:
# If Commanders already won CF #

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 1, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, same_conference_teams, other_conference_teams):
    # Conference Winner EV
    # 100% win for this team
    conference_winner_ev = payouts[winning_team]['conference']  # 100% win for this team

    # for Team X in other conference:
    # += (Probability of Team X Winning Conferene) * (Conference Winner Potential Payout for Team X)
    for team in other_conference_teams:
        conference_winner_ev += (
            team_probabilities[team]['semifinal_prob'] *
            payouts[team]['conference']
        )
    
    # Championship EV
    # 100% probability this team reaches championship
    # for Team X in other conference:
    # += (Prob of this team beating Team X in Champ.) * (Champ. Payout for this team) + (Prob. of Team X beating this team in Champ.) * (Champ. Payout for Team X)
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    # Total Portfolio EV
    return conference_winner_ev + championship_ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
scenarios = [
    ("kansas city chiefs", conference_a, conference_b),
    ("buffalo bills", conference_a, conference_b),
    ("washington commanders", conference_b, conference_a),
    ("philadelphia eagles", conference_b, conference_a),
]

ev_results = {}
for winner, same_conference, other_conference in scenarios:
    ev = calculate_portfolio_ev(winner, same_conference, other_conference)
    ev_results[f"{winner.title()} wins semifinal"] = ev

# print EV results
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:.2f}")


Portfolio EV if Kansas City Chiefs wins semifinal: $432089.00
Portfolio EV if Buffalo Bills wins semifinal: $235802.00
Portfolio EV if Washington Commanders wins semifinal: $341796.98
Portfolio EV if Philadelphia Eagles wins semifinal: $196374.03


# If Commanders already won CF:
Portfolio EV if Kansas City Chiefs wins semifinal: $432089.00
  Portfolio EV if Buffalo Bills wins semifinal: $235802.00


In [5]:
# If Eagles already won CF #

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 1, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, same_conference_teams, other_conference_teams):
    # Conference Winner EV
    # 100% win for this team
    conference_winner_ev = payouts[winning_team]['conference']  # 100% win for this team

    # for Team X in other conference:
    # += (Probability of Team X Winning Conferene) * (Conference Winner Potential Payout for Team X)
    for team in other_conference_teams:
        conference_winner_ev += (
            team_probabilities[team]['semifinal_prob'] *
            payouts[team]['conference']
        )
    
    # Championship EV
    # 100% probability this team reaches championship
    # for Team X in other conference:
    # += (Prob of this team beating Team X in Champ.) * (Champ. Payout for this team) + (Prob. of Team X beating this team in Champ.) * (Champ. Payout for Team X)
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    # Total Portfolio EV
    return conference_winner_ev + championship_ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
scenarios = [
    ("kansas city chiefs", conference_a, conference_b),
    ("buffalo bills", conference_a, conference_b),
    ("washington commanders", conference_b, conference_a),
    ("philadelphia eagles", conference_b, conference_a),
]

ev_results = {}
for winner, same_conference, other_conference in scenarios:
    ev = calculate_portfolio_ev(winner, same_conference, other_conference)
    ev_results[f"{winner.title()} wins semifinal"] = ev

# print EV results
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:.2f}")


Portfolio EV if Kansas City Chiefs wins semifinal: $265791.80
Portfolio EV if Buffalo Bills wins semifinal: $114883.60
Portfolio EV if Washington Commanders wins semifinal: $341796.98
Portfolio EV if Philadelphia Eagles wins semifinal: $196374.03


# If Eagles already won CF:
Portfolio EV if Kansas City Chiefs wins semifinal: $265791.80
  Portfolio EV if Buffalo Bills wins semifinal: $114883.60